In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import tcav
from src.download_data import download_imagenet, download_random_imagenet_classes
import pickle
from src.dataset import *
from src.concept_vectors import *
from src.util import *
import random
from src.plots import *
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine 
import pandas as pd
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs
from src.hierarchy import *
from src.metrics import *
import pickle
import tensorflow as tf
import keras
from keras.models import load_model, Model

2023-01-18 11:30:50.172739: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
mnist_attributes = get_mnist_attributes()

In [5]:
robustness_image_metric(create_linkage_hierarchy,load_tcav_vectors_simple,'mnist',mnist_attributes,[43,44,45])

46.666666666666664

In [6]:
responsiveness_image_metric(create_linkage_hierarchy,load_tcav_vectors_simple,'mnist',mnist_attributes,[43,44,45])

46.0

In [9]:
responsiveness_image_metric(create_linkage_hierarchy,
                                       load_label_vectors_simple,'mnist',mnist_attributes,[43,44,45])

47.0

In [11]:
sess = utils.create_session()

2023-01-18 10:35:02.110670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 10:35:02.118093: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
GRAPH_PATH = "./dataset/models/inception5h/tensorflow_inception_graph.pb"
LABEL_PATH = "./dataset/models/inception5h/imagenet_comp_graph_label_strings.txt"
mymodel = model.GoogleNetWrapper_public(sess,
                                    GRAPH_PATH,
                                    LABEL_PATH)

In [13]:
activation_dir = './results/activations'
working_dir = './results/tmp'
image_dir = "./dataset/images"

In [15]:
act_generator = act_gen.ImageActivationGenerator(mymodel, 
                                                 image_dir, activation_dir, max_examples=100)

In [18]:
d = act_generator.get_examples_for_concept("0_number")

In [19]:
d.shape

(100, 224, 224, 3)

In [21]:
activations = load_activations_tcav(mnist_attributes,experiment_name="mnist",max_examples=100)

2023-01-18 10:40:48.060535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [25]:
np.linalg.norm(activations["0_number"])

82407.38

In [5]:
hierarchy_method=create_linkage_hierarchy
embedding_method=load_label_vectors_simple
dataset='mnist'
attributes=mnist_attributes
random_seeds=[43,44,45]
model="GoogleNet"

In [6]:
random.seed(42)
    
n_concepts = len(attributes)//5
compare_concepts = len(attributes)//5

selected_concepts = random.sample(attributes,k=n_concepts)

avg_truthfulness = []

In [7]:
seed = 43
concept = "0_number"

In [8]:
activation_dir = './results/activations'
working_dir = './results/tmp'
image_dir = "./dataset/images"

max_images = 100

if dataset == 'mnist':
    concepts = get_mnist_attributes()

In [9]:
activations = load_activations_tcav(concepts,experiment_name=dataset,max_examples=max_images)

2023-01-18 11:31:35.994492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 11:31:36.002495: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-18 11:31:40.246849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [10]:
our_vectors = activations[concept]

In [11]:
cdist

<function scipy.spatial.distance.cdist(XA, XB, metric='euclidean', *, out=None, **kwargs)>

In [13]:
all_distance_pairs = []
for other_concept in concepts:
    print(other_concept)
    if other_concept != concept:
        distances = cdist(our_vectors,activations[other_concept],metric='cosine')
        all_distance_pairs.append((other_concept,np.mean(distances)))

0_color
0_number
1_color
1_number
2_color
2_number
3_color
3_number
4_color
4_number
5_color
5_number
6_color
6_number
7_color
7_number
8_color
8_number
9_color
9_number
spurious


In [14]:
all_distance_pairs = sorted(all_distance_pairs,key=lambda k: k[1])
[i[0] for i in all_distance_pairs][:5]

['0_color', '5_number', '6_number', '5_color', '6_color']

In [15]:
all_distance_pairs

[('0_color', 0.2703246392913351),
 ('5_number', 0.3634966880977542),
 ('6_number', 0.36597444109862576),
 ('5_color', 0.36602611221781145),
 ('6_color', 0.3692161675511568),
 ('9_number', 0.37112409359930537),
 ('8_number', 0.3792494928402702),
 ('9_color', 0.3808778023167217),
 ('8_color', 0.3813513889149904),
 ('3_number', 0.38222491701495953),
 ('spurious', 0.3834654850094098),
 ('3_color', 0.39138542483929173),
 ('2_number', 0.40489694623799805),
 ('2_color', 0.4084804230132554),
 ('4_number', 0.42284139339690574),
 ('1_color', 0.42325033832208264),
 ('7_color', 0.42465486258556595),
 ('4_color', 0.42499817839442877),
 ('1_number', 0.4272763922677328),
 ('7_number', 0.4313626597133339)]

In [16]:
co_occuring_concepts = find_similar_concepts(concept,dataset,model,compare_concepts,seed)

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
co_occuring_concepts_hierarchy = rank_distance_concepts(embedding_method,dataset,concept,
                                                                    co_occuring_concepts,seed)

In [22]:
co_occuring_concepts

['0_color', '6_color', '5_number', '5_color']

In [21]:
co_occuring_concepts_hierarchy

['0_color', '6_color', '5_number', '5_color']

In [26]:
stats.kendalltau(co_occuring_concepts,co_occuring_concepts_hierarchy).correlation

1.0

In [ ]:
truthfulness_metric(create_linkage_hierarchy,load_tcav_vectors_simple,
                    'mnist',mnist_attributes,[43,44,45])

In [38]:
hierarchy_method = create_linkage_hierarchy
embedding_method = load_tcav_vectors_simple
dataset = 'mnist'
attributes = mnist_attributes[:4]
random_seeds = [43,44,45]
suffix = '_image_responsiveness'

In [42]:
baseline_hierarchies = [create_hierarchy(hierarchy_method,embedding_method,dataset,attributes,seed) for seed in random_seeds]
robustness_hierarchies = [create_hierarchy(hierarchy_method,embedding_method,dataset+'_image_robustness',attributes,seed) for seed in random_seeds]
responsiveness_hierarchies = [create_hierarchy(hierarchy_method,embedding_method,dataset+'_image_responsiveness',attributes,seed) for seed in random_seeds]

In [43]:
responsiveness_hierarchies[0].distance(baseline_hierarchies[0])

4.0

In [44]:
robustness_hierarchies[0].distance(baseline_hierarchies[0])

4.0